In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error as MSE
import warnings
warnings.filterwarnings("ignore")

In [62]:
###
# ДЗ ML3:
# 1. Датасет titanic, разделить на train/test в пропорции 0.7/0.3
# 2. Предсказать возраст Age на основании других параметров (отберите сами какие)
# 3. Подобрать оптимальные гиперпараметры модели с помощью GridSearch, в идеале получить лучшее качество на test
###

In [63]:
## Резюме

# Результат линейной регрессии на тесте после оптимизации (MSE):  -175.53050966534144
# Лучшие параметры регрессии: {'fit_intercept': True, 'positive': False}

# Лучший показатель DecisionTreeRegressor на тесте при оптимизации (MSE):  -205.57545932291464
# Лучшие параметры дерева: {'max_depth': 2, 'min_samples_leaf': 9, 'min_samples_split': 2}

# Лучший показатель XGBRegressor на тесте при оптимизации (MSE):  -204.26866346743162
# Лушчие параметры XGB: {'colsample_bytree': 0.3, 'gamma': 0.0, 'max_depth': 5, 'min_child_weight': 7}

In [64]:
raw_data = pd.read_csv('https://raw.githubusercontent.com/RMA1011/da_hw/main/ML/HW%203/titanic.csv')
RANDOM_STATE = 42

In [65]:
#### Обработка данных
raw_data.dropna(inplace = True)
features = raw_data[['Survived','Pclass','Sex','SibSp','Fare']]
features['Sex'].replace({'male':0,'female':1}, inplace = True)
target = raw_data['Age']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = RANDOM_STATE)

In [71]:
#### Пробуем линейную регрессию
lin_mod = LinearRegression()
fit_mod = LinearRegression().fit(X = X_train,y = y_train)
pred_lin = fit_mod.predict(X_test)
print('Результат линейной регрессии на тесте до оптимизации (MSE): ', MSE(y_test, pred_lin))
parameters = {'fit_intercept': [True,False],
'positive':[True, False]}
lin_grid = GridSearchCV(estimator = lin_mod, scoring = 'neg_mean_squared_error',param_grid = parameters, cv = 5, n_jobs =-1, refit = True)
lin_grid = lin_grid.fit(X_train, y_train)
best_lin_model = lin_grid.best_estimator_
lin_pred = best_lin_model.predict(X_test)

print('Результат линейной регрессии на обучении после оптимизации (MSE): ', -1*lin_grid.best_score_)
print('Результат линейной регрессии на тесте после оптимизации (MSE): ', MSE(y_test, lin_pred))
print('Лучшие параметры: ', lin_grid.best_params_)

Результат линейной регрессии на тесте до оптимизации (MSE):  175.53050966534144
Результат линейной регрессии на обучении после оптимизации (MSE):  239.40141875015624
Результат линейной регрессии на тесте после оптимизации (MSE):  175.53050966534144
Лучшие параметры:  {'fit_intercept': True, 'positive': False}


In [ ]:
#### Пробуем дерево
decis_tree = DecisionTreeRegressor(random_state=42)
parameters_tree = {
    'max_depth':  [1,2,3,4,5,6,7,8,9,10],
    'min_samples_split': [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf': [2,3,4,5,6,7,8,9,10]
}

fit_tree = decis_tree.fit(X_train, y_train)
fit_tree_pred = fit_tree.predict(X_test)
print('Лучший показатель на обучении до оптимизации (MSE): ', MSE(y_test, fit_tree_pred))

decis_grid = GridSearchCV(
    estimator = decis_tree, 
    param_grid = parameters_tree,
    scoring = 'neg_mean_squared_error',
    cv = 5,
    refit = True     
)

decis_grid = decis_grid.fit(X_train, y_train)
print('Лучшие параметры: ',decis_grid.best_params_)
print('Лучший показатель на обучении при оптимизации (MSE): ', -1*decis_grid.best_score_)
best_decis_model = decis_grid.best_estimator_
best_decis_model = best_decis_model.fit(X_train,y_train)
best_predicted = best_decis_model.predict(X_test)
print('Лучший показатель на тесте при оптимизации (MSE): ', MSE(y_test, best_predicted))

Лучший показатель на обучении до оптимизации (MSE):  342.5092072727272
Лучшие параметры:  {'max_depth': 2, 'min_samples_leaf': 9, 'min_samples_split': 2}
Лучший показатель на обучении при оптимизации (MSE):  227.80952042290446
Лучший показатель на тесте при оптимизации (MSE):  205.57545932291464


In [ ]:
### Пробуем xgboost!

xgb_r = xgb.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 10, seed = RANDOM_STATE)

xgb_r.fit(X_train, y_train)
xgb_pred = xgb_r.predict(X_test)
print("Результат на тесте до оптимизации (MSE) : % f" %(MSE(y_test, xgb_pred)))

parameters_xgb = [
    {
    #'learning rate': (0.05,0.10,0.15),
    'max_depth': [3,4,5,6,8],
    'min_child_weight': [1,3,5,7],
    'gamma': [0.0,0.1,0.2],
    'colsample_bytree': [0.3, 0.4]
    }
]

xgb_grid = GridSearchCV(
    estimator = xgb_r, 
    param_grid = parameters_xgb,
    scoring = 'neg_mean_squared_error',
    cv = 5,
    refit = True)

xgb_grid = xgb_grid.fit(X_train, y_train)
print('Лучшие параметры: ',xgb_grid.best_params_)
print('Лучший показатель на обучении при оптимизации (MSE): ', -1*xgb_grid.best_score_)
best_xgb_model = xgb_grid.best_estimator_
best_xgb_model = best_xgb_model.fit(X_train,y_train)
xgb_opt_pred = best_xgb_model.predict(X_test)
print('Лучший показатель на тесте при оптимизации (MSE): ', MSE(y_test, xgb_opt_pred))

Результат на тесте до оптимизации (MSE) :  208.103283
Лучшие параметры:  {'colsample_bytree': 0.3, 'gamma': 0.0, 'max_depth': 5, 'min_child_weight': 7}
Лучший показатель на обучении при оптимизации (MSE):  228.82802988909128
Лучший показатель на тесте при оптимизации (MSE):  204.26866346743162
